はい、承知いたしました。アップロードされた`classified_order_history_with_account.csv`ファイル（Amazonの注文データ）を利用し、PythonでRAG（検索拡張生成）システムを構築するための、具体的な手順と初期のデータ加工コードをご提案します。

[cite_start]このデータには「Order ID」「Order Date」「書籍名」「価格 (数値)」「分類」「勘定科目」といった列が含まれており、質問応答の知識ベースとして非常に適しています[cite: 188]。

## 🐍 RAG構築の初期ステップ：データ準備（Python/pandas）

RAGの最初のステップは、構造化されたCSVデータから、AIが理解しやすい\*\*非構造化された文章（ドキュメント）\*\*を作成することです。

### ステップ 1: データの読み込みと準備

Pythonの`pandas`ライブラリを使って、ファイルを読み込みます。

In [ ]:
# 必要なライブラリを一括インストール
!pip install langchain chromadb sentence-transformers pandas langchain-community langchain-core openai langchain-anthropic

In [ ]:
# --- ファイル読み込み ---

import pandas as pd

# ファイル名
file_name = 'classified_order_history_with_account.csv'

# データの読み込み
try:
    df = pd.read_csv(file_name)
    print(f"データが正常に読み込まれました。行数: {len(df)}")
except FileNotFoundError:
    print(f"エラー: ファイル {file_name} が見つかりません。")
    exit()

# データの最初の5行を表示して確認
# print(df.head())

### ステップ 2: RAGドキュメントの作成（チャンキングと整形）

各行（注文）を、RAGの検索対象となる一つの独立したドキュメント（テキストの塊、または**チャンク**）に変換します。これにより、LLMは特定の質問に対して、この整形された文章を根拠として利用できます。

ここでは、`apply()`とラムダ関数を使ってドキュメントを作成します。

In [ ]:
# --- 静的フィルタ実装モデル ---

# RAGの次のステップ（概念コード、永続化モデル）

import os
import pandas as pd
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_core.documents import Document

# --- 0. 設定とデータの準備 ---
# PERSIST_DIR = "./chroma_db_orders" # ベクトルDBの保存先フォルダ
# 【変更】V2など、新しいフォルダ名に変更
PERSIST_DIR = "./chroma_db_orders_v2"
file_name = 'classified_order_history_with_account.csv'
df = pd.read_csv(file_name)

# 勘定科目のメタデータを含むドキュメントリストを準備
rag_documents = []
for index, row in df.iterrows():
    date_part = row['Order Date'].split('T')[0]
    document_content = (
        f"【注文データ】 日付: {date_part}, 商品名: {row['書籍名']}, "
        f"価格: {row['価格 (数値)']}円, 分類: {row['分類']}, "
        f"勘定科目: {row['勘定科目']}, 注文ID: {row['Order ID']}"
    )
    metadata = {
        "order_id": row['Order ID'],
        "order_date": date_part,
        "price": row['価格 (数値)'],
        "classification": row['分類'],
        "account_title": row['勘定科目']
    }
    rag_documents.append(Document(page_content=document_content, metadata=metadata))

# 3. 埋め込みモデルの初期化
# 速度と精度のバランスが取れたモデルを選択
embeddings = SentenceTransformerEmbeddings(model_name="intfloat/multilingual-e5-large")


# --- 4. ベクトルデータベースの作成 / ロード ---

# フォルダが存在するかチェック（以前に実行済みか）
if os.path.exists(PERSIST_DIR):
    print("✅ 既存のベクトルデータベースをロードしています...")

    # 既存のデータベースをロードする
    vectorstore = Chroma(
        persist_directory=PERSIST_DIR,
        embedding_function=embeddings
    )
    print(f"   ロード完了。データベース内のドキュメント数: {vectorstore._collection.count()}")

else:
    print("⏳ ベクトルデータベースを新規作成（埋め込み処理）しています...")

    # 新規作成し、指定したフォルダに保存する
    vectorstore = Chroma.from_documents(
        documents=rag_documents,
        embedding=embeddings,
        persist_directory=PERSIST_DIR
    )
    # 永続化を確定させる（ディスクに書き出す）
    vectorstore.persist()
    print("   新規作成と永続化が完了しました。")


# 5. Retriever（検索器）の設定（フィルタリング機能付き）
# 「研究開発費」のデータだけを確実に検索対象とする
filter_condition = {
    "$and": [
        {"account_title": "私用"},
        {"order_date": "2024-12-30"} # 日付による完全一致フィルタを追加
    ]
}

retriever = vectorstore.as_retriever(
    search_kwargs={
        "filter": filter_condition,
        "k": 50}
)

print("\nRAGのRetrieverの準備が完了しました。")
print("次回以降は、このコードを実行しても「埋め込み処理」はスキップされます。")

In [ ]:
# --- 動的フィルタ実装モデル ---

import os
import pandas as pd
import json
from typing import Optional, Dict, Any

# LangChain関連のインポート
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_core.documents import Document
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_anthropic import ChatAnthropic
from pydantic import BaseModel, Field

# --- 0. 設定とデータの準備 ---
# ※ DBを再構築する場合は、フォルダ名を変更してください
PERSIST_DIR = "./chroma_db_orders_v2"
file_name = 'classified_order_history_with_account.csv'

try:
    df = pd.read_csv(file_name)
    df['勘定科目'] = df['勘定科目'].str.strip() # データクリーニングを再実行
except FileNotFoundError:
    print("エラー: CSVファイルが見つかりません。ファイルをアップロードし直してください。")
    exit()

# ドキュメントリストの準備
rag_documents = []
for index, row in df.iterrows():
    date_part = row['Order Date'].split('T')[0]
    document_content = (
        f"【注文データ】 日付: {date_part}, 商品名: {row['書籍名']}, "
        f"価格: {row['価格 (数値)']}円, 分類: {row['分類']}, "
        f"勘定科目: {row['勘定科目']}, 注文ID: {row['Order ID']}"
    )
    metadata = {
        "order_id": row['Order ID'],
        "order_date": date_part,
        "price": row['価格 (数値)'],
        "classification": row['分類'],
        "account_title": row['勘定科目']
    }
    rag_documents.append(Document(page_content=document_content, metadata=metadata))


# 3. 埋め込みモデルの初期化 (【ここを intfloat/multilingual-e5-large に統一】)
embeddings = SentenceTransformerEmbeddings(model_name="intfloat/multilingual-e5-large")


# 4. ベクトルデータベースの作成 / ロード（永続化ロジック）
if os.path.exists(PERSIST_DIR):
    print("✅ 既存のベクトルデータベースをロードしています...")
    vectorstore = Chroma(
        persist_directory=PERSIST_DIR,
        embedding_function=embeddings
    )
else:
    print("⏳ ベクトルデータベースを新規作成（埋め込み処理）しています...")
    # ※ intfloat/multilingual-e5-large のDLとベクトル化に時間がかかります
    vectorstore = Chroma.from_documents(documents=rag_documents, embedding=embeddings, persist_directory=PERSIST_DIR)
    vectorstore.persist()
    print("   新規作成と永続化が完了しました。")


# =======================================================
# 【自動フィルタリングのロジック】
# =======================================================

# 1. Pydanticスキーマの定義
class FilterSchema(BaseModel):
    """ユーザーの質問から、検索に必要な勘定科目と日付を抽出するためのスキーマ"""
    account_title: Optional[str] = Field(None, description="質問に含まれる勘定科目を抽出してください。例: '私用', '研究開発費'。含まれていない場合はnull。")
    order_date: Optional[str] = Field(None, description="質問に含まれる日付を YYYY-MM-DD 形式で抽出してください。例: '2024-01-30'。含まれていない場合はnull。")

# 2. LLMの設定
llm = ChatAnthropic(temperature=0, model_name="claude-3-haiku-20240307")

# 3. フィルタ生成プロンプトの定義
FILTER_PROMPT = """
あなたは、ユーザーの質問を分析し、注文履歴DBのメタデータフィルタを生成する専門家です。
質問から、「勘定科目 (account_title)」と「日付 (order_date)」を抽出し、JSON形式で出力してください。
質問に情報が含まれていない場合は 'null' を使用してください。日付は必ず YYYY-MM-DD 形式にしてください。

質問: {question}
"""
filter_template = PromptTemplate.from_template(FILTER_PROMPT)

# 4. フィルタ生成LLMチェーンの作成
filter_generator_chain = filter_template | llm.with_structured_output(FilterSchema)

# 5. LLMの出力（Pydanticオブジェクト）をChromaDBフィルタ形式（$and）に変換する関数
def create_chroma_filter(filter_schema: FilterSchema) -> Dict[str, Any]:
    """PydanticスキーマオブジェクトからChromaDBの$andフィルタ形式を生成する"""
    filter_list = []

    if filter_schema.account_title:
        cleaned_title = filter_schema.account_title.strip()
        filter_list.append({"account_title": cleaned_title})
        print(f"-> 適用フィルタ: 勘定科目 = '{cleaned_title}'")

    if filter_schema.order_date:
        filter_list.append({"order_date": filter_schema.order_date})
        print(f"-> 適用フィルタ: 日付 = '{filter_schema.order_date}'")

    if filter_list:
        return {"$and": filter_list}
    else:
        print("-> フィルタ条件なし: 全ドキュメントに対して意味検索を実行します。")
        return {} # フィルタなし = 全ドキュメント検索

# 6. 動的Retrieverを定義する関数
def get_dynamic_retriever(filter_json: Dict[str, Any], k: int = 50):
    """動的にフィルタを適用したRetrieverを生成する"""
    return vectorstore.as_retriever(
        search_kwargs={
            "filter": filter_json,
            "k": k
        }
    )

# 7. エンドツーエンドのRAGチェーンの構築 (LCEL)
# 最終回答のためのプロンプト
ANSWER_PROMPT = """
あなたは、提供されたAmazon注文履歴データのエキスパートです。
以下のContextに示された注文履歴情報のみを根拠として、**必ず日本語で**正確に、そして簡潔に回答してください。
Contextに質問に答えるために必要な情報がない場合は、「該当する情報が見つかりませんでした」とだけ回答してください。

質問: {question}
---
Context: {context}
"""
answer_template = PromptTemplate.from_template(ANSWER_PROMPT)

# 最終回答チェーン
qa_chain = (
    answer_template
    | llm
)

# RAG全体のメインチェーン
final_rag_chain = (
    {
        # 1. フィルタを生成
        "filter_json": (
            RunnablePassthrough.assign(
                filter_schema=lambda x: filter_generator_chain.invoke({"question": x["question"]})
            )
            # 2. PydanticオブジェクトをChromaDBのフィルタ形式に変換
            | RunnableLambda(lambda x: create_chroma_filter(x["filter_schema"]))
        ),
        # 3. 質問をそのまま渡す (最終回答用)
        "question": RunnablePassthrough(),
    }
    # 4. フィルタJSONと質問を受け取り、動的Retrieverを生成し検索
    | RunnableLambda(lambda x: get_dynamic_retriever(x["filter_json"]).with_config(run_name="DynamicRetrieval").invoke(x["question"]))
    # 5. 検索結果と元の質問を結合し、最終回答チェーンに渡す
    | qa_chain
)

print("\n\n✅ intfloat/multilingual-e5-large を使用した自動フィルタリングRAGシステムが起動しました。")
print("これからは、フィルタ条件を意識せず、自然な日本語で質問してください。")

In [ ]:
import os
import pandas as pd
import json
from typing import Optional, Dict, Any, List

# LangChain関連のインポート
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_core.documents import Document
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import PromptTemplate
from langchain_anthropic import ChatAnthropic
from pydantic import BaseModel, Field

# --- 0. 設定とデータの準備 ---
# DBを再構築する場合に備え、新しいフォルダ名を推奨
PERSIST_DIR = "./chroma_db_orders_v2"
file_name = 'classified_order_history_with_account.csv'

# LLMの設定
llm = ChatAnthropic(temperature=0, model_name="claude-3-haiku-20240307")

try:
    # CSVファイルの読み込み
    df = pd.read_csv(file_name)
    # 勘定科目列のクリーニング（前後の空白文字除去）
    df['勘定科目'] = df['勘定科目'].str.strip()
except FileNotFoundError:
    print("エラー: CSVファイルが見つかりません。ファイルをアップロードし直してください。")
    exit()

# ドキュメントリストの準備
rag_documents = []
for index, row in df.iterrows():
    # 日付を YYYY-MM-DD 形式で抽出
    date_part = row['Order Date'].split('T')[0]

    # ドキュメント本文（検索対象となるテキスト）
    document_content = (
        f"【注文データ】 日付: {date_part}, 商品名: {row['書籍名']}, "
        f"価格: {row['価格 (数値)']}円, 分類: {row['分類']}, "
        f"勘定科目: {row['勘定科目']}, 注文ID: {row['Order ID']}"
    )

    # メタデータ（フィルタリングキー）
    metadata = {
        "order_id": row['Order ID'],
        "order_date": date_part, # YYYY-MM-DD形式
        "price": row['価格 (数値)'],
        "account_title": row['勘定科目'] # クリーニング済み
    }
    rag_documents.append(Document(page_content=document_content, metadata=metadata))


# 3. 埋め込みモデルの初期化
embeddings = SentenceTransformerEmbeddings(model_name="intfloat/multilingual-e5-large")


# 4. ベクトルデータベースの作成 / ロード（永続化ロジック）
if os.path.exists(PERSIST_DIR):
    print("✅ 既存のベクトルデータベースをロードしています...")
    vectorstore = Chroma(
        persist_directory=PERSIST_DIR,
        embedding_function=embeddings
    )
    # ドキュメント数が表示されると安心感が増します
    print(f"   ロード完了。データベース内のドキュメント数: {vectorstore._collection.count()}")
else:
    print("⏳ ベクトルデータベースを新規作成（埋め込み処理）しています... (intfloat/multilingual-e5-largeのDL/ベクトル化に時間がかかります)")
    vectorstore = Chroma.from_documents(documents=rag_documents, embedding=embeddings, persist_directory=PERSIST_DIR)
    vectorstore.persist()
    print("   新規作成と永続化が完了しました。")


# =======================================================
# 【自動フィルタリングのロジック】
# =======================================================

# 1. Pydanticスキーマの定義
class FilterSchema(BaseModel):
    """ユーザーの質問から、検索に必要な勘定科目と日付を抽出するためのスキーマ"""
    account_title: Optional[str] = Field(None, description="質問に含まれる勘定科目を抽出してください。例: '私用', '研究開発費'。含まれていない場合はnull。")
    order_date: Optional[str] = Field(None, description="質問に含まれる日付を YYYY-MM-DD 形式で抽出してください。例: '2024-01-30'。含まれていない場合はnull。")

# 2. フィルタ生成プロンプトとチェーンの定義
FILTER_PROMPT = """
あなたは、ユーザーの質問を分析し、Amazon注文履歴データベースのメタデータフィルタを生成する専門家です。
ユーザーの質問から、メタデータとして利用できる「勘定科目」と「日付」を抽出し、JSON形式で出力してください。

- 勘定科目 (account_title) が質問に含まれていない場合は 'null' を使用してください。
- 日付 (order_date) が質問に含まれていない場合は 'null' を使用してください。
- 日付は必ず YYYY-MM-DD 形式にしてください。

質問: {question}
"""
filter_template = PromptTemplate.from_template(FILTER_PROMPT)

# フィルタ生成LLMチェーンの作成
filter_generator_chain = filter_template | llm.with_structured_output(FilterSchema)

# 3. LLMの出力（Pydanticオブジェクト）をChromaDBフィルタ形式（$and）に変換する関数
def create_chroma_filter(filter_schema: FilterSchema) -> Dict[str, Any]:
    """PydanticスキーマオブジェクトからChromaDBの$andフィルタ形式を生成する"""
    filter_list = []

    if filter_schema.account_title:
        cleaned_title = filter_schema.account_title.strip()
        filter_list.append({"account_title": cleaned_title})
        print(f"-> 適用フィルタ: 勘定科目 = '{cleaned_title}'")

    if filter_schema.order_date:
        filter_list.append({"order_date": filter_schema.order_date})
        print(f"-> 適用フィルタ: 日付 = '{filter_schema.order_date}'")

    if filter_list:
        # フィルタが一つでもあれば $and で結合して返す
        return {"$and": filter_list}
    else:
        # フィルタ条件がない場合は、空の辞書を返す（全件検索と同じ意味）
        print("-> フィルタ条件なし: 全ドキュメントに対して意味検索を実行します。")
        return {}

# 4. 動的Retrieverを定義する関数
def get_dynamic_retriever(filter_json: Dict[str, Any], k: int = 145): # 最大件数に近づけて設定
    """動的にフィルタを適用したRetrieverを生成する"""
    return vectorstore.as_retriever(
        search_kwargs={
            "filter": filter_json,
            "k": k # 検索件数は余裕を持って設定
        }
    )

# 5. 【★修正で追加された関数★】 ドキュメントリストを単一の文字列にフォーマットする関数
def format_docs(docs: List[Document]) -> str:
    """List[Document]をLLMプロンプト用の文字列に変換する"""
    # 検索結果が空の場合に対応
    if not docs:
        return "該当する注文履歴データは見つかりませんでした。"
    # ドキュメントのpage_contentを連結し、Contextとして整形
    return "\n\n---\n\n".join(doc.page_content for doc in docs)

# 6. 最終回答プロンプトとチェーン
ANSWER_PROMPT = """
あなたは、提供されたAmazon注文履歴データのエキスパートです。
以下のContextに示された注文履歴情報のみを根拠として、**必ず日本語で**正確に、そして簡潔に回答してください。
もし、Contextに質問に答えるために必要な情報が全く含まれていなかった場合は、「該当する情報が見つかりませんでした」とだけ回答してください。

質問: {question}
---
Context: {context}
"""
answer_template = PromptTemplate.from_template(ANSWER_PROMPT)

# 最終回答チェーン
qa_chain = (
    answer_template
    | llm
)

# 7. 【★修正されたメインチェーン★】 final_rag_chain
final_rag_chain = (
    {
        # 1. フィルタを生成
        "filter_json": (
            RunnablePassthrough.assign(
                filter_schema=lambda x: filter_generator_chain.invoke({"question": x["question"]})
            )
            | RunnableLambda(lambda x: create_chroma_filter(x["filter_schema"]))
        ),
        # 2. 質問をそのまま渡す
        "question": RunnablePassthrough(),
    }
    # 3. 検索を実行し、結果を 'context' キーに格納 (List[Document])
    | RunnablePassthrough.assign(
        context=RunnableLambda(
            lambda x: get_dynamic_retriever(x["filter_json"]).invoke(x["question"])
        )
    )
    # 4. 'context' を List[Document] から単一の文字列にフォーマット (エラー解消の核心)
    | RunnablePassthrough.assign(
        context=lambda x: format_docs(x["context"])
    )
    # 5. 最終回答チェーンに渡す
    | qa_chain
)

print("\n\n✅ 自動フィルタリングRAGシステムがエラーを修正し、最終起動しました。")
print("これからは、フィルタ条件を意識せず、自然な日本語で質問してください。")


# --- 質問の実行テスト ---

# 成功していた質問（日付と勘定科目）
question_ok = "2024年12月30日の勘定科目が私用の注文について、注文IDと商品名をすべて教えてください。"
print(f"\n--- 質問1（複合フィルタ）: {question_ok} ---")
result_ok = final_rag_chain.invoke({"question": question_ok})
print(f"--- 回答 ---\n{result_ok.content}")


# 失敗していた質問（単一フィルタ）
question_ng = "勘定科目が「新聞図書費」の注文に含まれる商品名を全てリストアップしてください。"
print(f"\n--- 質問2（単一フィルタ）: {question_ng} ---")
result_ng = final_rag_chain.invoke({"question": question_ng})
print(f"--- 回答 ---\n{result_ng.content}")

### ステップ 4: 質問応答の実行

ユーザーの質問を受け付け、LLMに回答を生成させます。

In [ ]:
# 質問を変数に代入
question_ok = "2024年12月30日の勘定科目が私用の注文について、注文IDと商品名をすべて教えてください。"
question_ng = "勘定科目が「新聞図書費」の注文に含まれる商品名を全てリストアップしてください。"

# 実行
result_ok = final_rag_chain.invoke({"question": question_ok})
result_ng = final_rag_chain.invoke({"question": question_ng})

In [ ]:
# --- 質問応答の実行（概念コード） ---

# 【修正点1】インポート元を langchain_core に変更
import os
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_anthropic import ChatAnthropic

In [ ]:
# --- APIキーの設定 ---
from google.colab import userdata

# Secretsからキーを取得
api_key = userdata.get('ANTHROPIC_API_KEY')
os.environ["ANTHROPIC_API_KEY"] = api_key  # ライブラリが環境変数を参照する場合

# 【変更点2】Claude LLMの初期化
llm = ChatAnthropic(
    model="claude-3-haiku-20240307", # 使用したいモデル名を指定
    temperature=0  # 回答のランダム性をゼロに設定
)

# プロンプトの定義（変更なし）
prompt_template = """
あなたは、提供されたAmazon注文履歴データのエキスパートです。
以下の{context}に示された注文履歴情報のみを根拠として、ユーザーの質問に正確に、そして簡潔に回答してください。
文脈に情報がない場合は、「この注文履歴データには情報がありません」と回答してください。

質問: {question}
---
文脈: {context}
"""

prompt = ChatPromptTemplate.from_template(prompt_template)

# RAGチェーンの構築（変更なし）
rag_chain = (
    # ここでは、retrieverは既に定義され、ベクトルDBがロードされていると仮定
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm    # ここでClaudeが使用される
    | StrOutputParser()
)

In [ ]:
# --- 質問の実行 ---
question = "2024年12月30日の勘定科目が私用の注文について、注文IDと商品名をすべて教えてください。"
result = rag_chain.invoke(question)
print(result)

In [ ]:
# --- 質問の実行 ---
question = "勘定科目が「新聞図書費」の注文に含まれる商品名を全てリストアップしてください。"
result = rag_chain.invoke(question)
print(result)

■ 以下過去コード



---



In [ ]:
# --- チャンキングとデータ整形 ---

# データをRAGのドキュメント形式に整形する関数
def create_rag_document(row):
    """DataFrameの1行をRAG用のドキュメント（テキスト）に変換する"""

    # Order Date は '2024-12-30T11:34:59Z' のような形式なので、日付部分のみ抽出
    date_part = row['Order Date'].split('T')[0]

    # 注文情報の詳細を、LLMが理解しやすい自然言語の文章としてまとめる
    document = (
        f"【注文データ】\n"
        f"  日付: {date_part}\n"
        f"  商品名: {row['書籍名']}\n"
        f"  価格: {row['価格 (数値)']}円\n"
        f"  分類: {row['分類']}\n"
        f"  勘定科目: {row['勘定科目']}\n"
        f"  注文ID: {row['Order ID']}\n"
    )
    return document

# 新しい列 'rag_document' にドキュメントを作成
df['rag_document'] = df.apply(create_rag_document, axis=1)

# 作成されたドキュメントの例を表示
# print("\n--- 作成されたRAGドキュメントの例（最初の1件） ---")
# print(df['rag_document'].iloc[0])

In [ ]:
# --- RAGの次のステップ（概念コード） ---

# 1. ライブラリのインポート
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_core.documents import Document

# 2. ドキュメントオブジェクトの準備
documents = [Document(page_content=doc) for doc in df['rag_document']]

# 3. 埋め込みモデルの初期化
# embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# 【推奨】多言語対応のモデルに変更（日本語の文脈理解に強い）
# 2.2GB
# embeddings = SentenceTransformerEmbeddings(model_name="intfloat/multilingual-e5-large")
# 1.1GB
embeddings = SentenceTransformerEmbeddings(model_name="paraphrase-multilingual-mpnet-base-v2")

# 4. ベクトルデータベースの作成
vectorstore = Chroma.from_documents(documents, embeddings)

# 5. Retriever（検索器）の設定
retriever = vectorstore.as_retriever()